In [1]:
# Extract images from Zip file and move to 'images' folder
!mkdir images
!unzip images.zip
!mv *.png images/

Archive:  images.zip
  inflating: images/BPF_length170.0_length2120.0_height20.0_sep20.0.png  
  inflating: images/BPF_length170.0_length2120.0_height20.0_sep35.0.png  
  inflating: images/BPF_length170.0_length2120.0_height20.0_sep50.0.png  
  inflating: images/BPF_length170.0_length2120.0_height20.0_sep65.0.png  
  inflating: images/BPF_length170.0_length2120.0_height20.0_sep80.0.png  
  inflating: images/BPF_length170.0_length2120.0_height25.0_sep20.0.png  
  inflating: images/BPF_length170.0_length2120.0_height25.0_sep35.0.png  
  inflating: images/BPF_length170.0_length2120.0_height25.0_sep50.0.png  
  inflating: images/BPF_length170.0_length2120.0_height25.0_sep65.0.png  
  inflating: images/BPF_length170.0_length2120.0_height25.0_sep80.0.png  
  inflating: images/BPF_length170.0_length2120.0_height30.0_sep20.0.png  
  inflating: images/BPF_length170.0_length2120.0_height30.0_sep35.0.png  
  inflating: images/BPF_length170.0_length2120.0_height30.0_sep50.0.png  
  inflating: imag

In [77]:
# Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset, random_split
from torch.amp import GradScaler, autocast
import pandas as pd
from PIL import Image
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import re
import os
from tqdm import tqdm

# Set Environment variable for Cuda
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [78]:
# Define a custom dataset#
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)    # Read annotations CSV file
        self.img_dir = img_dir                      # Image directory Path
        self.transform = transform                  # Image transformer (for manipulating image sizes)

    def __len__(self):
        return len(self.annotations) # Size of annotations file

    def __getitem__(self, index):
        img_name = self.annotations.iloc[index, 0]                          # Get the name of image at index
        image = Image.open(f"{self.img_dir}/{img_name}").convert('RGB')     # Open the image and convert to RGB if not already
        if self.transform:
            image = self.transform(image)                                   # Transform image (resize and change to Tensor type)

        frequencies = self.annotations.iloc[index, 1:].values.astype(float) # Extract annotations (frequency response values)
        return image, torch.tensor(frequencies, dtype=torch.float32)        # Return the image and it's annotation (Both as tensors)

In [79]:
# Define Global Variables
image_width = 450
image_height = 300
img_dir = './images'
csv_file = './annotations.csv'
num_outputs = 381

# Define image transformer (changes input image dimensions and changes them to tensors)
transform = transforms.Compose([
    transforms.Resize((image_height, image_width)),
    transforms.ToTensor(),
])

# Create and load dataset
dataset = CustomDataset(csv_file=csv_file, img_dir=img_dir, transform=transform)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [80]:
# Calculating number of input neurons for the linear fully-connected layer
def calculate_linears(num_layers: int, dim: list, conv_filter_size: int, conv_stride: int, maxp_filter_size: int, maxp_stride: int, padding: int) -> list:
    output = [dim[0], dim[1]]  # Initialize with the input dimensions

    for i in range(num_layers):
        # Calculate convolution output dimensions
        conv_output_width = ((output[0] - conv_filter_size + (2 * padding)) / conv_stride) + 1
        conv_output_height = ((output[1] - conv_filter_size + (2 * padding)) / conv_stride) + 1

        # Update output to convolution result
        output = [conv_output_width, conv_output_height]

        # Calculate max pooling output dimensions
        pool_output_width = ((output[0] - maxp_filter_size) / maxp_stride) + 1
        pool_output_height = ((output[1] - maxp_filter_size) / maxp_stride) + 1

        # Update output to pooling result
        output = [int(pool_output_width), int(pool_output_height)]

    return output  # Return final dimensions

In [130]:
# Circuit Convolution neural network configuration
input_layers = 3
Conv1_output_layers = 32
Conv2_output_layers = Conv1_output_layers * 2
Conv3_output_layers = Conv2_output_layers * 2
conv_filter_size = 3
conv_stride = 1
maxp_filter_size = 2
maxp_stride = 2
padding = 1
linear_layers = calculate_linears(num_layers = 3,
                                    dim = [image_width, image_height],
                                    conv_filter_size = conv_filter_size,
                                    conv_stride = conv_stride,
                                    maxp_filter_size = maxp_filter_size,
                                    maxp_stride = maxp_stride,
                                    padding = padding
                                    )
conv_flatten_widths = linear_layers[0]
conv_flatten_heights = linear_layers[1]
Lin1_output_layers = 768
Lin2_output_layers = Lin1_output_layers * 2 // 3 # 512
Lin3_output_layers = Lin2_output_layers // 2 # 256
dropout_prob = 0.05
conv_flat_output = conv_flatten_widths * conv_flatten_heights * Conv3_output_layers

In [117]:
# Circuit Convolution neural network configuration
input_layers = 3
Conv1_output_layers = 32
Conv2_output_layers = Conv1_output_layers * 2
Conv3_output_layers = Conv2_output_layers * 2
conv_filter_size = 3
conv_stride = 1
maxp_filter_size = 2
maxp_stride = 2
padding = 1
linear_layers = calculate_linears(num_layers = 3,
                                    dim = [image_width, image_height],
                                    conv_filter_size = conv_filter_size,
                                    conv_stride = conv_stride,
                                    maxp_filter_size = maxp_filter_size,
                                    maxp_stride = maxp_stride,
                                    padding = padding
                                    )
conv_flatten_widths = linear_layers[0]
conv_flatten_heights = linear_layers[1]
Lin1_output_layers = 1024
Lin2_output_layers = Lin1_output_layers 
Lin3_output_layers = Lin2_output_layers
Lin4_output_layers = Lin3_output_layers // 2
Lin5_output_layers = Lin4_output_layers // 2
dropout_prob = 0.05
conv_flat_output = conv_flatten_widths * conv_flatten_heights * Conv3_output_layers

In [131]:
class CircuitFrequencyResponseModel(nn.Module):
    def __init__(self, output_length):
        super(CircuitFrequencyResponseModel, self).__init__()

        # Convolutional layers for feature extraction
        self.conv1 = nn.Conv2d(input_layers, Conv1_output_layers, kernel_size=conv_filter_size, padding=padding)
        self.conv2 = nn.Conv2d(Conv1_output_layers, Conv2_output_layers, kernel_size=conv_filter_size, padding=padding)
        self.conv3 = nn.Conv2d(Conv2_output_layers, Conv3_output_layers, kernel_size=conv_filter_size, padding=padding)
        self.pool = nn.MaxPool2d(kernel_size=maxp_filter_size, stride=maxp_stride)

        self.fc1 = nn.Linear(conv_flat_output, Lin1_output_layers)
        self.fc2 = nn.Linear(Lin1_output_layers, Lin2_output_layers)
        self.fc3 = nn.Linear(Lin2_output_layers, Lin3_output_layers)
        # self.fc4 = nn.Linear(Lin3_output_layers, Lin4_output_layers)
        # self.fc5 = nn.Linear(Lin4_output_layers, Lin5_output_layers)
        self.dropout = nn.Dropout(p=dropout_prob)
        self.fc_out = nn.Linear(Lin3_output_layers, output_length)

        self.output_length = output_length

    def forward(self, x):
        # Forward pass through conv layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten feature map
        x = x.view(x.size(0), -1)

        # Forward pass through fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        # x = self.dropout(F.relu(self.fc4(x)))
        # x = self.dropout(F.relu(self.fc5(x)))
        x = self.fc_out(x)

        return x

In [132]:
# Training Variables (Define the model, loss function, and optimizer)
model = CircuitFrequencyResponseModel(output_length=num_outputs)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

# Move model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs=30

In [120]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, targets = images.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.squeeze(-1), targets)  # Compute loss across the sequence
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("Training complete!")

Epoch 1/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:42<00:00,  3.71it/s]


Epoch [1/30], Loss: 15.6180


Epoch 2/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:40<00:00,  3.86it/s]


Epoch [2/30], Loss: 12.1967


Epoch 3/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:40<00:00,  3.86it/s]


...

Epoch [28/30], Loss: 1.6040


Epoch 29/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:42<00:00,  3.71it/s]


Epoch [29/30], Loss: 1.6116


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:40<00:00,  3.86it/s]

Epoch [30/30], Loss: 1.6859
Training complete!


In [121]:
# Save model
try:
  torch.save(model.state_dict(), f'custom_cnn_model_{version_count}.pth')
  version_count += 1
  print("Model saved successfully!")
except Exception as e:
  version_count = 0
  torch.save(model.state_dict(), f'custom_cnn_model_{version_count}.pth')
  version_count += 1

Model saved successfully!


In [134]:
# Code for Output Prediction:
import pandas as pd
import matplotlib.pyplot as plt

def predict_output(model, image_path, transform):
    model.eval()
    with torch.no_grad():
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0)

        output = model(image)

        if output is not None:
            predicted_outputs = output.cpu().numpy().flatten()
            return predicted_outputs
        else:
            return None

def choose_random_file(directory):
    # Get a list of files in the directory
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

    # Check if there are any files in the directory
    if not files:
        return None

    # Choose a random file
    return random.choice(files)

def plot_predictions_vs_real(image_name, predicted_values):
    """
    Plots predicted values vs real values for a given image.

    Parameters:
    - image_name (str): The name of the image (e.g., 'image1.jpg').
    - predicted_values (array-like): Array of predicted values corresponding to the image.
    """

    # Load the CSV file containing annotations
    annotations_df = pd.read_csv(csv_file)

    # Find the row corresponding to the image name
    image_row = annotations_df[annotations_df['Image Name'] == image_name]

    if image_row.empty:
        print(f"No entry found for image: {image_name}")
        return

    # Assuming the real values are stored in columns named 'real_value_1', 'real_value_2', etc.
    real_values = image_row.iloc[:, 1:].values.flatten()  # This grabs all columns except 'image_name'

    # Check if predicted_values match the length of real_values
    if len(predicted_values) != len(real_values):
        print("Mismatch between predicted values and real values length.")
        return

    # Calculate the total difference (sum of absolute differences)
    total_difference = sum(abs(real_values[i] - predicted_values[i]) for i in range(len(real_values)))
    # print(f"Total difference between real and predicted values: {total_difference}")
    # print(f"Average difference between real and predicted values: {total_difference/len(real_values)}")

    # Plotting the graph
    # plt.figure(figsize=(10, 6))

    # # Plot real values (assumed to be in 'real_values')
    # plt.plot(real_values, label='Real Values', color='blue', marker='o', linestyle='-', markersize=5)

    # # Plot predicted values
    # plt.plot(predicted_values, label='Predicted Values', color='red', marker='x', linestyle='--', markersize=5)

    # # Add labels and title
    # plt.title(f"Real vs Predicted Values for {image_name}")
    # plt.xlabel('Index')
    # plt.ylabel('Value')

    # # Add a legend
    # plt.legend()

    # # Show the plot
    # plt.grid(True)
    # plt.show()
    return total_difference, total_difference/len(real_values)

In [139]:
model_path=f'./custom_cnn_model_{version_count - 2}.pth' # NN model path
net = CircuitFrequencyResponseModel(output_length=num_outputs) # Initialize the NN
net.load_state_dict(torch.load(model_path, weights_only=True, map_location=torch.device('cpu'))) # Load only the weights from the NN
t0 = 0
a0 = 0
check_range = 300
for i in range(check_range):
  test_image = choose_random_file("./images")
  image_path = f'./images/{test_image}'
  predicted_values = predict_output(net, image_path, transform) # Get predicted values
  t, a = plot_predictions_vs_real(test_image, predicted_values)
  if t > t0:
    t0 = t
  a0 += a
print(f"Worst Case single frequency discrepancy: {t0/num_outputs}")
print(f"Average single frequency discrepancy across {check_range} samples: {a0/check_range}")

Worst Case single frequency discrepancy: 6.680728474018669
Average single frequency discrepancy across 300 samples: 2.0687172468528527
